<a href="https://colab.research.google.com/github/laquita44/big-data-challenge/blob/main/shoe_ipynb_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [50]:
# For connection to Postgres run the following code 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-04-09 00:20:01--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-04-09 00:20:02 (10.3 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [51]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


In [52]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://shoe-inventory-1.s3.us-west-2.amazonaws.com/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

# Show Dataframe
shoes_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
shoes_df.show()



+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...|2015-01-31 00:08:00|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...| 

In [54]:
# Count the number of records (rows) in the dataset
shoes_df.count()


4366916

In [55]:
# check data type for schema
shoes_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [68]:
# Transform the dataset to fit the tables in the schema file. Be sure the DataFrames match in data type and in column name
from pyspark.sql.types import TimestampType, IntegerType

shoes_df = shoes_df.withColumn("customer_id", shoes_df["customer_id"].cast(IntegerType()))\
       .withColumn("product_parent", shoes_df["product_parent"].cast(IntegerType()))\
       .withColumn("review_date", shoes_df["review_date"].cast(TimestampType()))\
       .withColumn("star_rating", shoes_df["star_rating"].cast(IntegerType()))\
       .withColumn("helpful_votes", shoes_df["helpful_votes"].cast(IntegerType()))\
       .withColumn("total_votes", shoes_df["total_votes"].cast(IntegerType()))\

In [69]:
# Create DataFrame to match Schema table "review_id_table"
review_id_table = shoes_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R3P2HIOQCIN5ZU|   18069663|B000XB31C0|     265024781|2015-01-31 00:08:00|
|R12VVR0WH5Q24V|   16251825|B00CFYZH5W|     259035853|2015-01-31 00:08:00|
| RNCCKB6TV5EEF|   20381037|B00S8JNN3Q|     666066660|2015-01-31 00:08:00|
|R2NZXYIVCGB13W|     108364|B00XFBPOQG|     448483263|2015-01-31 00:08:00|
|R2EQ1TG9IT3OEQ|   45449350|B00SW64Y9W|       7853171|2015-01-31 00:08:00|
|R1WXA9JSC2H1U1|   19324665|B011F9E6LI|      14311457|2015-01-31 00:08:00|
|R12ENYLFGGNWRV|   50073594|B00HAUP1OI|     264821602|2015-01-31 00:08:00|
|R2R07E5PNXEUO5|   21706057|B00L1RKOKW|     767118055|2015-01-31 00:08:00|
|R27BA52AKWMWN3|   13708216|B005WA9MSE|     813856438|2015-01-31 00:08:00|
| RLF8DOID2KD5O|   40542649|B00BEYQI5C|     661491213|2015-01-31 00:08:00|
|R369CEXHXHC6NQ|   134094

In [70]:
# Create DataFrame to match Schema table "products"
products = shoes_df.select(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000XB31C0|Minnetonka Men's ...|
|B00CFYZH5W|Teva Men's Pajaro...|
|B00S8JNN3Q|Anne Klein Perfec...|
|B00XFBPOQG|adidas Men's 10K ...|
|B00SW64Y9W|OverBling Sneaker...|
|B011F9E6LI|MESSI 15.3 FG/AG ...|
|B00HAUP1OI|Hoka One One Men'...|
|B00L1RKOKW|Olukai Nohea Mesh...|
|B005WA9MSE|Carolina Mens 6" ...|
|B00BEYQI5C|Alegria Women's S...|
|B00EYAFTCQ|Naturalizer Women...|
|B00I0QHY32|Forever Link Wome...|
|B010FZZKYA|Versace Collectio...|
|B00V8B30K2|Twisted Girl's Ch...|
|B00LAVB1TC|Travel Smart Hand...|
|B003C1P8B0|Saucony Originals...|
|B000W3UL7W|Dr. Marten's Wome...|
|B00LX65PQO|Skechers Performa...|
|B00BEE7N3I|Hi-Tec Kid's Nepa...|
|B008NCHMBW|Foot Sox Original...|
+----------+--------------------+
only showing top 20 rows



In [71]:
#  To create customer_counts that was included in the Schema did a groupby
customers = shoes_df.groupby("customer_id").agg({
    "customer_id": "count"
    }).withColumnRenamed("count(customer_id)", "customer_count")
customers.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16121565|             5|
|   48146680|             1|
|   11062912|             1|
|   51451778|             1|
|   27920838|             1|
|    4919528|             3|
|   47802851|             2|
|   42560427|             1|
|   29467780|             1|
|   49703087|            12|
|   38456994|             4|
|     557836|             1|
|   43783459|             1|
|   23607725|             1|
|   45632184|             1|
|   19979069|             2|
|   28380959|             1|
|   46563381|             1|
|   21628739|             3|
|   23042837|             2|
+-----------+--------------+
only showing top 20 rows



In [72]:
# Create DataFrame to match Schema table "vine_table"
vine_table = shoes_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|
|R12VVR0WH5Q24V|          5|            0|          0|   N|
| RNCCKB6TV5EEF|          4|            0|          0|   N|
|R2NZXYIVCGB13W|          5|            0|          6|   N|
|R2EQ1TG9IT3OEQ|          3|            0|          0|   N|
|R1WXA9JSC2H1U1|          5|            1|          1|   N|
|R12ENYLFGGNWRV|          5|            1|          1|   N|
|R2R07E5PNXEUO5|          4|            0|          0|   N|
|R27BA52AKWMWN3|          5|            0|          0|   N|
| RLF8DOID2KD5O|          3|            0|          0|   N|
|R369CEXHXHC6NQ|          5|            0|          0|   N|
|R171PPIJXFONVI|          5|            0|          0|   N|
|R2JDNM8F2P06FU|          4|            1|          1|   N|
|R2W977FO4M97XT|          5|            

In [76]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://myshoepostgresdb.cgqnrqyjvdcw.us-west-2.rds.amazonaws.com:5432/<database-name>"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [ ]:
# Write tables into an RDS instance for "review_id_table","products","customers","vine_table"

In [62]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [73]:
# Drop duplicate values for table "product"
products = products.dropDuplicates()

In [65]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [66]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [67]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)